# Classification Level 1 

In [1]:
import joblib
import os
processed_path = "../data/processed/"
# Load data
X = joblib.load(os.path.join(processed_path, "X_scaled.joblib"))
y_bin = joblib.load(os.path.join(processed_path, "y_lvl1.joblib"))
# --- Level 1: Binary Classification ---
# Use X and y_bin directly
# --- Level 2: Attack Classification ---
# Load encoded labels
#y_multi = joblib.load(os.path.join(processed_path, "y_lvl2_encoded.joblib"))
# Filter in memory (No extra storage needed!)
#X_attack = X[y_bin == 1]
#y_attack = y_multi[y_bin == 1]